# Mức 2: Đánh giá mô hình deraining (pretrained) trên dữ liệu ngoài bài báo
**CHÚ Ý:**
- Bật GPU T4 x2 trước khi chạy notebook
- Thêm các Kaggle dataset sau làm input:
    - For pre-trained weight: https://kaggle.com/datasets/48e9d5bc84d59d1aaee28d770c6ceae05bcbc151efe901d644ab2a354afae48b
    - For test set: https://kaggle.com/datasets/5400c159acc0837a0cbf3834149bc5e03004a42c8bc22245927af84233815c56

# Môi trường & Dependencies
- Kaggle

In [1]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.3 LTS"
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [2]:
# !rm -rf /kaggle/working/*

In [3]:
# !sudo apt-get install libtinfo5

In [4]:
!python --version
!python3.8 --version

Python 3.10.12
/bin/bash: line 1: python3.8: command not found


In [5]:
# !sudo apt update
# !sudo apt install python3.8 python3.8-distutils
# !sudo apt install python3.8-dev

In [6]:
!python --version
!python3.8 --version

Python 3.10.12
/bin/bash: line 1: python3.8: command not found


In [7]:
# %cd /kaggle
# !wget https://bootstrap.pypa.io/get-pip.py
# !sudo python3.8 get-pip.py

In [8]:
!pip --version
!pip3.8 --version

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
/bin/bash: line 1: pip3.8: command not found


In [9]:
# !python3.8 -m pip install torch==1.8.1 torchvision==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

In [10]:
!pip show torch torchvision

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, easyocr, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, sentence-transformers, stable-baselines3, timm, torchaudio, torchmetrics, torchvision
---
Name: torchvision
Version: 0.20.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: easyocr, fastai, timm


In [11]:
# !pip3.8 install tensorboard einops scikit-image pytorch_msssim opencv-python
!pip install tensorboard einops scikit-image pytorch_msssim opencv-python

In [12]:
!pip show tensorboard einops scikit-image pytorch_msssim opencv-python

Name: tensorboard
Version: 2.17.1
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, grpcio, markdown, numpy, packaging, protobuf, setuptools, six, tensorboard-data-server, werkzeug
Required-by: tensorflow
---
Name: einops
Version: 0.8.0
Summary: A new flavour of deep learning operations
Home-page: https://github.com/arogozhnikov/einops
Author: Alex Rogozhnikov
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 
---
Name: scikit-image
Version: 0.25.0
Summary: Image processing in Python
Home-page: https://scikit-image.org
Author: 
Author-email: 
License: Files: *
         Copyright: 2009-2022 the scikit-image team
         License: BSD-3-Clause
         
         Files: doc/source/themes/scikit-image/layout.html
         Copyright: 2007-2010 

# Copy source code

In [13]:
!cp -r "/kaggle/input/sfnet-source-code-and-model/SFNet" \
    "/kaggle/working/"

In [14]:
!cp "/kaggle/input/youtube-frames-for-deraining/test_derain_youtube.py" \
    "/kaggle/working/SFNet/Image_deraining/"

In [15]:
!ls /kaggle/working

SFNet


# Kiểm tra dữ liệu đã thêm vào Kaggle Input

In [16]:
!ls /kaggle/input/sfnet-source-code-and-model

deraining.pkl		  fixed_main_for_deraining.py  test_derain_muc_2b.py
deraining_runtime_weight  SFNet			       test_derain_muc_2.py


In [17]:
!ls /kaggle/input/youtube-frames-for-deraining

extracted_from_videos  test_derain_youtube.py


# Copy dữ liệu từ Kaggle Input

In [18]:
!mkdir -p /kaggle/working/test_data/

In [19]:
!cp -r "/kaggle/input/youtube-frames-for-deraining/extracted_from_videos/extracted_from_videos/"* \
    "/kaggle/working/test_data/"

In [20]:
!ls "/kaggle/working/test_data/"

video1	video2	video3	video4


In [21]:
# Create temporary ground truth images
import os

data_dir = "/kaggle/working/test_data/"
test_sets = ["video1", "video2", "video3", "video4"]

for test_set in test_sets:
    input_dir = os.path.join(data_dir, test_set, "input")
    target_dir = os.path.join(data_dir, test_set, "target")

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".png"):
            # Create a temporary ground truth image
            temp_gt_path = os.path.join(target_dir, filename)
            
            # Copy the input image to the target directory
            input_path = os.path.join(input_dir, filename)
            
            os.system(f"cp {input_path} {temp_gt_path}")

            print(f"Created fake ground truth image: {temp_gt_path}")

Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_15.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_16.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_3.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_13.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_20.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_9.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_1.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_17.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_2.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_14.png
Created fake ground truth image: /kaggle/working/test_data/video1/target/frame_4.png
Created fake ground truth image: /kaggle/working/test_data/

# Inference

In [22]:
# # %cd /content/drive/MyDrive/advanced_machine_learning/colab/SFNet/Motion_deblurring
# %cd /kaggle/SFNet
# %cd pytorch-gradual-warmup-lr/
# # !python3.8 setup.py install
# !python setup.py install
# # !python3.10 setup.py install
# %cd ..

In [23]:
# !cp /kaggle/SFNet/Image_dehazing/eval.py /kaggle/SFNet/Image_deraining
# !cp /kaggle/SFNet/Motion_deblurring/eval.py /kaggle/SFNet/Image_deraining

In [24]:
!mkdir /kaggle/working/SFNet/Image_deraining/results

In [25]:
!mkdir -p /kaggle/working/paper_pretrained_weight_results/
!mkdir -p /kaggle/working/our_fine_tuned_weight_results/

In [26]:
%cd /kaggle/working/SFNet/Image_deraining
%pwd
%ls

/kaggle/working/SFNet/Image_deraining
data/             models/    test_derain_youtube.py  utils.py
deraining_test.m  README.md  test.py                 valid.py
main.py           results/   train.py


In [27]:
!python test_derain_youtube.py --test_model /kaggle/input/sfnet-source-code-and-model/deraining.pkl \
    --data_dir /kaggle/working/test_data/ 

/kaggle/working/SFNet/Image_deraining/test_derain_youtube.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(args.test_model)
results/SFNet/deraini

In [28]:
!mv /kaggle/working/SFNet/Image_deraining/results /kaggle/working/paper_pretrained_weight_results/

In [35]:
!rm -rf /kaggle/working/SFNet/Image_deraining/results/*
!mkdir /kaggle/working/SFNet/Image_deraining/results

In [36]:
!python test_derain_youtube.py --test_model /kaggle/input/sfnet-source-code-and-model/deraining_runtime_weight/model_420.pkl \
    --data_dir /kaggle/working/test_data/ 

/kaggle/working/SFNet/Image_deraining/test_derain_youtube.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(args.test_model)
results/SFNet/deraini

In [37]:
!mv /kaggle/working/SFNet/Image_deraining/results /kaggle/working/our_fine_tuned_weight_results/

# Download dữ liệu
- Kết quả đã được upload tại link: https://studenthcmusedu-my.sharepoint.com/:u:/g/personal/24c11071_student_hcmus_edu_vn/EWSPaAJNn_VAhU-uE39hGvsBGCv1ttneuctYAZjejSkqxQ?e=aNiZdc

In [38]:
%cd /kaggle/working/

/kaggle/working


In [42]:
# Zip kết quả inference
!zip -r deraining_youtube_test_output.zip "paper_pretrained_weight_results" "our_fine_tuned_weight_results"

updating: paper_pretrained_weight_results/ (stored 0%)
updating: paper_pretrained_weight_results/results/ (stored 0%)
updating: paper_pretrained_weight_results/results/SFNet/ (stored 0%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/ (stored 0%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/video3/ (stored 0%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/video3/frame_15.png (deflated 1%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/video3/frame_16.png (deflated 1%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/video3/frame_3.png (deflated 1%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/video3/frame_13.png (deflated 1%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/video3/frame_20.png (deflated 0%)
updating: paper_pretrained_weight_results/results/SFNet/deraining/video3/frame_9.png (deflated 1%)
updating: paper_pretrained_weight_results/results/

In [47]:
%cd /kaggle/working/
!ls

/kaggle/working
deraining_youtube_test_output.zip  paper_pretrained_weight_results  test_data
our_fine_tuned_weight_results	   SFNet
